## A Large Language Model-based tool to facilitate data harmonization: feature analysis

In [ ]:
#****************************************
# MIT License
# Copyright (c) 2025 Zexu Li, Jinying Chen
#  
# author(s): Zexu Li, Jinying Chen, Boston University Chobanian & Avedisian School of Medicine
# date: 2025-7-7
# ver: 1.0
# 
# This code was written to support data analysis for the Data Harmonization Using Natural Language 
# Processing (NLP harmonization) project and the 2025 paper published in PLOS One.
# The code is for research use only, and is provided as it is.
# 

## Including permutation importance calculation, paired t-test on 50 trails

In [ ]:
import os
import re
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score,f1_score
from sklearn.metrics import roc_curve, precision_recall_curve, auc, make_scorer, recall_score, accuracy_score, precision_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold
import ast
import numpy as np
import scipy.stats as stats
import pickle

In [ ]:
# Load input data for feature analysis
datadir = "[path to input data]"
Final_df_ML = pd.read_csv(datadir+'ML_dataset_021825_v3.csv')

# path to ML outputs from 50 trials
# used to extract train/test split information by the function main_perm
folder_path = datadir + "/02232025res_v2/" 

In [ ]:
def calculate_mean_difference_and_ci(column1, column2, confidence_level=0.95):
    """
    Calculate the mean difference and 95% confidence interval for the difference in means
    between two columns of values.
    
    Parameters:
    column1 (list or np.array): First column of values
    column2 (list or np.array): Second column of values
    confidence_level (float): Confidence level for the interval (default is 0.95 for 95% CI)
    
    Returns:
    tuple: mean difference, (lower bound of CI, upper bound of CI)
    """
    # Convert the lists to numpy arrays
    column1 = np.array(column1)
    column2 = np.array(column2)

    # Calculate the means
    mean1 = np.mean(column1)
    mean2 = np.mean(column2)

    # Calculate the standard deviations
    std1 = np.std(column1, ddof=1)  # Using ddof=1 to get the sample standard deviation
    std2 = np.std(column2, ddof=1)

    # Calculate the sample sizes
    n1 = len(column1)
    n2 = len(column2)

    # Calculate the standard error for the difference in means
    sem = np.sqrt((std1**2 / n1) + (std2**2 / n2))

    # Calculate the degrees of freedom
    df = n1 + n2 - 2

    # Calculate the critical value from the t-distribution for the given confidence interval
    critical_value = stats.t.ppf((1 + confidence_level) / 2, df)

    # Calculate the margin of error
    margin_of_error = critical_value * sem

    # Calculate the confidence interval
    mean_difference = mean1 - mean2
    confidence_interval = (mean_difference - margin_of_error, mean_difference + margin_of_error)

    return mean_difference, confidence_interval


def extract_between_phrases(file_path, start_phrase, end_phrase):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    start_index = -1
    end_index = -1
    for i, line in enumerate(lines):
        if line.startswith(start_phrase):
            start_index = i
        if end_phrase in line:
            end_index = i
            break
    
    if start_index != -1 and end_index != -1:
        return ''.join([line.strip() for line in lines[start_index:end_index+1]])
    else:
        return []
    
def extract_train_sources(text):
    # Use regex to find the 'Train Source' and 'Test Source' parts
    train_match = re.search(r"Train Source:\[(.*?)\], Test Source", text)
    
    if train_match:
        train_sources = train_match.group(1)
        # Split the sources by separating them with ' ' and remove any empty strings
        train_sources_list = [source for source in train_sources.split("'") if source.strip()]
        if ', ' in train_sources_list:
            train_sources_list = [item for item in train_sources_list if item != ', ']
        return train_sources_list
    else:
        return ""
    
    
def extract_lines(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        relevant_lines = [line.strip() for line in lines if line.strip().startswith('Best Grid')]
        return relevant_lines
    
def select_true_and_false(df_source):
    true_values = df_source[df_source['Mapping_result'] == 1]
    false_values = df_source[df_source['Mapping_result'] == 0].sample(n=200*len(true_values), random_state=42)
    return pd.concat([true_values, false_values])

In [ ]:
#main function to generate the permutation result
#Permutation importance

def main_perm(Final_df_ML):
    result = {}
    #result_SKIpermu = {}
    file_num = 0
    ALL_permu_importance_dict30 = {}
    ALL_permu_mean_importance_dict30 = {}
    ALL_permu_importance_dict20 = {}
    ALL_permu_mean_importance_dict20 = {}
    ALL_permu_importance_dict10 = {}
    ALL_permu_mean_importance_dict10 = {}
    ALL_permu_importance_dict5 = {}
    ALL_permu_mean_importance_dict5 = {}
    ALL_permu_importance_dictMRR = {}
    ALL_permu_mean_importance_dictMRR = {}
    
    test_res_full = {}
    
    for file_name in os.listdir(folder_path):
        file_num+=1
        if file_name.endswith('.txt') and file_name.startswith('output'):
            file_path = os.path.join(folder_path, file_name)
            text = extract_between_phrases(file_path,'Train Source','Test Source')
            train_sources = extract_train_sources(text)
            #print(train_source)
            
            relevant_lines = extract_lines(file_path)
            print(f"Numbers extracted from {file_name}:")
            
            grids = {}
            i = 0
            for line in relevant_lines:
                #print(line)
                if line.startswith('Best Grid'):
                    i+=1
                    Grid = re.search(r"Best Grid:\{(.*?)\}, unique_HR", line)
                    grids[i] = Grid.group(1)
    
            train = Final_df_ML[Final_df_ML['Source'].isin(train_sources)]
            test = Final_df_ML[~Final_df_ML['Source'].isin(train_sources)]
            test_copy = test.copy()
        
        
            train_dataonly = train.groupby('Source').apply(select_true_and_false).reset_index(drop=True)
            
            train_dataonly = train_dataonly[['miniLM_on_label', 'e5_on_label', 'mpnet_on_label', 'fuzzy_on_label','biolord_on_label',
                           'miniLM_on_label_key', 'e5_on_label_key', 'mpnet_on_label_key', 'fuzzy_on_label_key','biolord_on_label_key',
                            'miniLM_on_sheet', 'e5_on_sheet', 'mpnet_on_sheet', 'fuzzy_on_sheet','biolord_on_sheet',
                               'Label_len_EU','deriv_info_null_EU','deriv_info_len_EU', 'deriv_info_null_JP','deriv_info_len_JP',
                               'Label_len_JP','Mapping_result']]
            test_dataonly = test[['miniLM_on_label', 'e5_on_label', 'mpnet_on_label', 'fuzzy_on_label','biolord_on_label',
                           'miniLM_on_label_key', 'e5_on_label_key', 'mpnet_on_label_key', 'fuzzy_on_label_key','biolord_on_label_key',
                            'miniLM_on_sheet', 'e5_on_sheet', 'mpnet_on_sheet', 'fuzzy_on_sheet','biolord_on_sheet',
                               'Label_len_EU','deriv_info_null_EU','deriv_info_len_EU', 'deriv_info_null_JP','deriv_info_len_JP',
                               'Label_len_JP','Mapping_result']]
            
            X_train = train_dataonly.drop('Mapping_result',axis = 1)
            y_train = train_dataonly[['Mapping_result']].values.ravel()
            X_test = test_dataonly.drop('Mapping_result',axis = 1)
            y_test = test_dataonly[['Mapping_result']].values.ravel()
            
            
            
            
            rf = RandomForestClassifier(random_state=42)
            print('Full model complete')
            dict_string = "{" + grids.get(3) + "}"
            # Convert the string to a dictionary
            g = ast.literal_eval(dict_string)
            rf.set_params(**g)
            rf.fit(X_train,y_train)
            
            
            feature_importances = rf.feature_importances_
            # Create a DataFrame to hold feature names and their importance scores
            features_df = pd.DataFrame({
                'Feature': X_train.columns,
                'Importance': feature_importances
            })

            # Sort the DataFrame by importance scores in descending order
            ranked_features_df = features_df.sort_values(by='Importance', ascending=False)
            result[file_num] = ranked_features_df
            
            
            #full
            y_pred = rf.predict(X_test)
            y_pred_proba = rf.predict_proba(X_test)[:, 1]
            test_copy['probability'] = y_pred_proba
            test_copy['rank'] = test_copy.groupby('Source')['probability'].rank(ascending=False)
            positive = test_copy[test_copy['Mapping_result'] == 1]
            unique_positive = positive.groupby('Source')['rank'].min().reset_index()
            top_30_HR_unique = len(unique_positive[unique_positive['rank'] <=30])/len(unique_positive)
            top_20_HR_unique = len(unique_positive[unique_positive['rank'] <=20])/len(unique_positive)
            top_10_HR_unique = len(unique_positive[unique_positive['rank'] <=10])/len(unique_positive)
            top_5_HR_unique = len(unique_positive[unique_positive['rank'] <=5])/len(unique_positive)
            max_ranks = positive.groupby('Source')['rank'].idxmin()
            result_df = positive.loc[max_ranks]
            result_df['MRR'] = 1/result_df['rank']
            MRR = sum(result_df['MRR'])/ len(result_df)
            test_res_full[file_num] = [top_30_HR_unique,top_20_HR_unique,top_10_HR_unique,top_5_HR_unique,MRR]
            
            
            
            permu_importance_dict30 = {}
            permu_mean_importance_dict30 = {}
            permu_importance_dict20 = {}
            permu_mean_importance_dict20 = {}
            permu_importance_dict10 = {}
            permu_mean_importance_dict10 = {}
            permu_importance_dict5 = {}
            permu_mean_importance_dict5 = {}
            permu_importance_dictMRR = {}
            permu_mean_importance_dictMRR = {}
            #Permutation imp from algorithm
            for columns_name in X_test.columns:
                print(columns_name)
                top_30_HR_permu_list = []
                top_20_HR_permu_list = []
                top_10_HR_permu_list = []
                top_5_HR_permu_list = []
                MRR_permu_list = []
                for repeat_num in range(20):
                    test_for_permu = test.copy()
                    X_test_shuffle = X_test.copy()#deep copy
                    X_test_shuffle[columns_name] = np.random.RandomState(seed=repeat_num).permutation(X_test_shuffle[columns_name].values)# np.random.RandomState(seed=repeat_num).permutation(10)
                    y_pred_proba_permu = rf.predict_proba(X_test_shuffle)[:, 1]
                    test_for_permu['probability'] = y_pred_proba_permu
                    test_for_permu['rank'] = test_for_permu.groupby('Source')['probability'].rank(ascending=False)
                    positive_permu = test_for_permu[test_for_permu['Mapping_result'] == 1]
                    unique_positive_permu = positive_permu.groupby('Source')['rank'].min().reset_index()
                    top_30_HR_unique_permu = len(unique_positive_permu[unique_positive_permu['rank'] <=30])/len(unique_positive_permu)
                    top_20_HR_unique_permu = len(unique_positive_permu[unique_positive_permu['rank'] <=20])/len(unique_positive_permu)
                    top_10_HR_unique_permu = len(unique_positive_permu[unique_positive_permu['rank'] <=10])/len(unique_positive_permu)
                    top_5_HR_unique_permu = len(unique_positive_permu[unique_positive_permu['rank'] <=5])/len(unique_positive_permu)
                    top_30_HR_permu_list.append(top_30_HR_unique - top_30_HR_unique_permu)
                    top_20_HR_permu_list.append(top_20_HR_unique - top_20_HR_unique_permu)
                    top_10_HR_permu_list.append(top_10_HR_unique - top_10_HR_unique_permu)
                    top_5_HR_permu_list.append(top_5_HR_unique - top_5_HR_unique_permu)
                    
                    max_ranks_permu = positive_permu.groupby('Source')['rank'].idxmin()
                    result_df_permu = positive_permu.loc[max_ranks_permu]
                    result_df_permu['MRR'] = 1/result_df_permu['rank']
                    MRR_permu = sum(result_df_permu['MRR'])/ len(result_df_permu)
                    MRR_permu_list.append(MRR - MRR_permu)
                permu_importance_dict30[columns_name] = top_30_HR_permu_list
                permu_mean_importance_dict30[columns_name] = np.mean(top_30_HR_permu_list)
                permu_importance_dict20[columns_name] = top_20_HR_permu_list
                permu_mean_importance_dict20[columns_name] = np.mean(top_20_HR_permu_list)
                permu_importance_dict10[columns_name] = top_10_HR_permu_list
                permu_mean_importance_dict10[columns_name] = np.mean(top_10_HR_permu_list)
                permu_importance_dict5[columns_name] = top_5_HR_permu_list
                permu_mean_importance_dict5[columns_name] = np.mean(top_5_HR_permu_list)
                permu_importance_dictMRR[columns_name] = MRR_permu_list
                permu_mean_importance_dictMRR[columns_name] = np.mean(MRR_permu_list)
            ALL_permu_importance_dict30[file_num] = permu_importance_dict30
            ALL_permu_mean_importance_dict30[file_num] = permu_mean_importance_dict30
            ALL_permu_importance_dict20[file_num] = permu_importance_dict20
            ALL_permu_mean_importance_dict20[file_num] = permu_mean_importance_dict20
            ALL_permu_importance_dict10[file_num] = permu_importance_dict10
            ALL_permu_mean_importance_dict10[file_num] = permu_mean_importance_dict10
            ALL_permu_importance_dict5[file_num] = permu_importance_dict5
            ALL_permu_mean_importance_dict5[file_num] = permu_mean_importance_dict5
            ALL_permu_importance_dictMRR[file_num] = permu_importance_dictMRR
            ALL_permu_mean_importance_dictMRR[file_num] = permu_mean_importance_dictMRR
           
            
    return result,ALL_permu_importance_dict30,ALL_permu_mean_importance_dict30,ALL_permu_importance_dict20,ALL_permu_mean_importance_dict20,ALL_permu_importance_dict10,ALL_permu_mean_importance_dict10,ALL_permu_importance_dict5,ALL_permu_mean_importance_dict5 ,ALL_permu_importance_dictMRR,ALL_permu_mean_importance_dictMRR,test_res_full

In [ ]:
def calculate_mean_across_dicts(dict_list):
    """
    Calculate the mean value for each key across multiple dictionaries.

    Parameters:
    dict_list (list of dict): A list of dictionaries with the same keys.

    Returns:
    dict: A dictionary with keys and their corresponding mean values.
    """
    # Initialize sum and count dictionaries
    sums = {}
    counts = {}

    # Iterate through each dictionary
    for d in dict_list.values():
        for key, value in d.items():
            if key not in sums:
                sums[key] = 0
                counts[key] = 0
            sums[key] += value
            counts[key] += 1

    # Calculate the mean for each key
    means = {key: sums[key] / counts[key] for key in sums}
    df = pd.DataFrame(list(means.items()), columns=['key', 'mean'])

    # Add a rank column
    df['rank'] = df['mean'].rank(ascending=False, method='min')

    # Sort the DataFrame by rank
    df = df.sort_values(by='rank')

    return df.reset_index(drop=True)

def rank_feq_5(row):
    n = 0
    for i in rank_list:
        if row[i]<=5: 
            n+=1
    return n/50

def rank_feq_10(row):
    n = 0
    for i in rank_list:
        if row[i]<=10: 
            n+=1
    return n/50

In [ ]:
result,ALL_permu_importance_dict30,ALL_permu_mean_importance_dict30,ALL_permu_importance_dict20,ALL_permu_mean_importance_dict20,ALL_permu_importance_dict10,ALL_permu_mean_importance_dict10,ALL_permu_importance_dict5,ALL_permu_mean_importance_dict5 ,ALL_permu_importance_dictMRR,ALL_permu_mean_importance_dictMRR,test_res_full = main_perm(Final_df_ML)

In [ ]:
with open(datadir + 'ALL_permu_mean_importance_dict5.pkl', 'wb') as f:
    pickle.dump(ALL_permu_mean_importance_dict5, f)

with open(datadir + 'ALL_permu_mean_importance_dict10.pkl', 'wb') as f:
    pickle.dump(ALL_permu_mean_importance_dict10, f)

with open(datadir + 'ALL_permu_mean_importance_dictMRR.pkl', 'wb') as f:
    pickle.dump(ALL_permu_mean_importance_dictMRR, f)

In [ ]:
#process the hr-5 permutation result (average over 20 random seed)
import pickle
file_path = datadir + 'ALL_permu_mean_importance_dict5.pkl'
with open(file_path, 'rb') as file:
    ALL_permu_mean_importance_dict5 = pickle.load(file)
HR5 = calculate_mean_across_dicts(ALL_permu_mean_importance_dict5)

HR5_avg_df = pd.DataFrame.from_dict(ALL_permu_mean_importance_dict5, orient='index').T
for i in HR5_avg_df.columns:
    rank_name = str(i)+'_rank'
    HR5_avg_df[rank_name] = HR5_avg_df[i].rank(ascending= False,method='min')
    

importance_list = []
rank_list = []
for i in range(1,51):
    imp = i
    rank =  str(i) +'_rank'
    importance_list.append(imp)
    rank_list.append(rank)
HR5_avg_df['Avg_importance'] = HR5_avg_df[importance_list].sum(axis = 1)/50
HR5_avg_df['Avg_rank'] = HR5_avg_df[rank_list].sum(axis = 1)/50



HR5_avg_df['top_10_feq'] = HR5_avg_df.apply(rank_feq_10,axis = 1)
HR5_avg_df['top_5_feq'] = HR5_avg_df.apply(rank_feq_5,axis = 1)
#HR5_avg_df.to_csv('HR5_avg_df.csv',index = False)

In [ ]:
HR5_res = HR5_avg_df[['Avg_importance','Avg_rank','top_10_feq','top_5_feq']].rename(columns=lambda x: f"{x}{'_HR5'}")
HR5_res

In [ ]:
#Hr10 result
import pickle
file_path = datadir + 'ALL_permu_mean_importance_dict10.pkl'
with open(file_path, 'rb') as file:
    ALL_permu_mean_importance_dict10 = pickle.load(file)
HR10 = calculate_mean_across_dicts(ALL_permu_mean_importance_dict10)

HR10_avg_df = pd.DataFrame.from_dict(ALL_permu_mean_importance_dict10, orient='index').T
for i in HR10_avg_df.columns:
    rank_name = str(i)+'_rank'
    HR10_avg_df[rank_name] = HR10_avg_df[i].rank(ascending= False,method='min')
    

importance_list = []
rank_list = []
for i in range(1,51):
    imp = i
    rank =  str(i) +'_rank'
    importance_list.append(imp)
    rank_list.append(rank)
HR10_avg_df['Avg_importance'] = HR10_avg_df[importance_list].sum(axis = 1)/50
HR10_avg_df['Avg_rank'] = HR10_avg_df[rank_list].sum(axis = 1)/50

def rank_feq_5(row):
    n = 0
    for i in rank_list:
        if row[i]<=5: 
            n+=1
    return n/50

def rank_feq_10(row):
    n = 0
    for i in rank_list:
        if row[i]<=10: 
            n+=1
    return n/50

HR10_avg_df['top_10_feq'] = HR10_avg_df.apply(rank_feq_10,axis = 1)
HR10_avg_df['top_5_feq'] = HR10_avg_df.apply(rank_feq_5,axis = 1)
#HR10_avg_df.to_csv('HR10_avg_df.csv',index = False)

In [ ]:
#MRR result
import pickle
file_path = datadir + 'ALL_permu_mean_importance_dictMRR.pkl'
with open(file_path, 'rb') as file:
    ALL_permu_mean_importance_dictMRR = pickle.load(file)
MRR = calculate_mean_across_dicts(ALL_permu_mean_importance_dictMRR)

MRR_avg_df = pd.DataFrame.from_dict(ALL_permu_mean_importance_dictMRR, orient='index').T
for i in MRR_avg_df.columns:
    rank_name = str(i)+'_rank'
    MRR_avg_df[rank_name] = MRR_avg_df[i].rank(ascending= False,method='min')
    

importance_list = []
rank_list = []
for i in range(1,51):
    imp = i
    rank =  str(i) +'_rank'
    importance_list.append(imp)
    rank_list.append(rank)
MRR_avg_df['Avg_importance'] = MRR_avg_df[importance_list].sum(axis = 1)/50
MRR_avg_df['Avg_rank'] = MRR_avg_df[rank_list].sum(axis = 1)/50

def rank_feq_5(row):
    n = 0
    for i in rank_list:
        if row[i]<=5: 
            n+=1
    return n/50

def rank_feq_10(row):
    n = 0
    for i in rank_list:
        if row[i]<=10: 
            n+=1
    return n/50

MRR_avg_df['top_10_feq'] = MRR_avg_df.apply(rank_feq_10,axis = 1)
MRR_avg_df['top_5_feq'] = MRR_avg_df.apply(rank_feq_5,axis = 1)
#MRR_avg_df.to_csv('MRR_avg_df.csv',index = False)

In [ ]:
MRR_res = MRR_avg_df[['Avg_importance','Avg_rank','top_10_feq','top_5_feq']].rename(columns=lambda x: f"{x}{'_MRR'}")
HR10_res = HR10_avg_df[['Avg_importance','Avg_rank','top_10_feq','top_5_feq']].rename(columns=lambda x: f"{x}{'_HR10'}")
HR5_res = HR5_avg_df[['Avg_importance','Avg_rank','top_10_feq','top_5_feq']].rename(columns=lambda x: f"{x}{'_HR5'}")